In [ ]:
%%writefile huffman_encoding.cu
#include <stdio.h>
#include <cuda.h>

// CUDA kernel for dummy encoding
__global__ void encodeHuffman(char *input, char *output, int *codes, int length) {
    int idx = threadIdx.x;
    if (idx < length) {
        output[idx] = (char)(codes[(int)input[idx]]);
    }
}

int main() {
    const int length = 10;
    char h_input[] = "HELLOCUDA!";     // input string
    char h_output[length+1];           // output string
    h_output[length] = '\0';           // null-terminator

    int h_codes[128]; // Dummy Huffman code table

    // Map ASCII codes to uppercase alphabets cyclically: 'A' to 'Z'
    for (int i = 0; i < 128; i++) {
        h_codes[i] = 'A' + (i % 26);
    }

    char *d_input, *d_output;
    int *d_codes;

    cudaMalloc((void**)&d_input, length * sizeof(char));
    cudaMalloc((void**)&d_output, length * sizeof(char));
    cudaMalloc((void**)&d_codes, 128 * sizeof(int));

    cudaMemcpy(d_input, h_input, length * sizeof(char), cudaMemcpyHostToDevice);
    cudaMemcpy(d_codes, h_codes, 128 * sizeof(int), cudaMemcpyHostToDevice);

    encodeHuffman<<<1, length>>>(d_input, d_output, d_codes, length);

    cudaMemcpy(h_output, d_output, length * sizeof(char), cudaMemcpyDeviceToHost);

    printf("Original : %s\\n", h_input);
    printf("Encoded  : %s\\n", h_output);

    printf("Original ASCII : ");
    for (int i = 0; i < length; i++)
        printf("%d ", (int)h_input[i]);
    printf("\\n");

    printf("Encoded ASCII  : ");
    for (int i = 0; i < length; i++)
        printf("%d ", (int)h_output[i]);
    printf("\\n");

    cudaFree(d_input);
    cudaFree(d_output);
    cudaFree(d_codes);

    return 0;
}


Writing huffman_encoding.cu


In [ ]:
!nvcc huffman_encoding.cu -o huffman
!./huffman


Original : HELLOCUDA!\nEncoded  : \nOriginal ASCII : 72 69 76 76 79 67 85 68 65 33 \nEncoded ASCII  : 0 0 0 0 0 0 0 0 0 0 \n